In [7]:
import pandas as pd
import altair as alt
import numpy as np
import sqlite3

# Create DataBase and Add Table

In [12]:
conn = sqlite3.connect('Netflix.db')

cur = conn.cursor()

# cur.execute("""CREATE TABLE nflx (
#     date string,
#     low float,
#     open float,
#     volume integer,
#     high float,
#     close float
# )""")

conn.commit()
conn.close()

# Fill Table

In [24]:
path = 'C:/Users/Blake Dennett/Downloads/Spring2023/appliedProgramming/Data/stock_market_data/sp500/csv/j-p/nflx.csv'
df = pd.read_csv(path)
new_row = []
for r in df.iterrows():
    row = r[1].tolist()
    new_row.append(row[0:5])
    # add row into db
    for r in row:
        print(r)
    

23-05-2002
1.1457140445709229
1.1564290523529053
104790000
1.2428569793701172
1.1964290142059326
1.1964290142059326
24-05-2002
1.1971429586410522
1.214285969734192
11104800
1.225000023841858
1.2100000381469729
1.2100000381469729
28-05-2002
1.157142996788025
1.2135709524154663
6609400
1.2321430444717407
1.157142996788025
1.157142996788025
29-05-2002
1.0857139825820925
1.1642860174179075
6757800
1.1642860174179075
1.1035710573196411
1.1035710573196411
30-05-2002
1.0714290142059326
1.1078569889068604
10154200
1.1078569889068604
1.0714290142059326
1.0714290142059326
31-05-2002
1.0714290142059326
1.0785709619522097
8464400
1.0785709619522097
1.076429009437561
1.076429009437561
